In [1]:
%load_ext bigdata
%pig_start
%timeout 300
!hadoop fs -rm data.tsv .
!hadoop fs -put data.tsv .
!hadoop fs -ls

Deleted data.tsv
rm: `.': Is a directory
Found 2 items
-rw-r--r--   1 root supergroup       1780 2020-02-16 16:55 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-02-16 15:48 truck_event_text_partition.csv


In [2]:
%%pig
records = LOAD 'data.tsv' AS (c1:CHARARRAY, c2:CHARARRAY, c3:CHARARRAY);
DUMP records;

 records = LOAD 'data.tsv' AS (c1:CHARARRAY, c2:CHARARRAY, c3:CHARARRAY);
 DUMP records
2020-02-16 16:56:22,883 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:56:23,251 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-02-16 16:56:23,258 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-02-16 16:56:23,276 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2020-02-16 16:56:23,919 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2020-02-16 16:56:23,938 [JobControl] INF

In [3]:
%%pig
third_col = FOREACH records GENERATE c3;
third_col_b = FOREACH third_col GENERATE REPLACE(c3, '\\]', '') AS c3;
third_col_c = FOREACH third_col_b GENERATE REPLACE(c3, '\\[', '') AS c3;
third_col_d = FOREACH third_col_c GENERATE REPLACE(c3, '\\#', '') AS c3;
third_col_e = FOREACH third_col_d GENERATE REPLACE(c3, '[0-9]', '') AS c3;
DUMP third_col_e;

 third_col = FOREACH records GENERATE c3;
 third_col_b = FOREACH third_col GENERATE REPLACE(c3, '\\]', '') AS c3;
 third_col_c = FOREACH third_col_b GENERATE REPLACE(c3, '\\[', '') AS c3;
 third_col_d = FOREACH third_col_c GENERATE REPLACE(c3, '\\#', '') AS c3;
;third_col_e = FOREACH third_col_d GENERATE REPLACE(c3, '[0-9]', '') AS c3 
 DUMP third_col_e;
2020-02-16 16:58:52,157 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:58:52,860 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:58:52,898 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 16:58:52,923 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 16:58:52,984 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmit

In [4]:
%%pig
div = FOREACH third_col_e GENERATE FLATTEN(TOKENIZE(c3)) AS c3;
grouped = GROUP div BY c3;
letcount = FOREACH grouped GENERATE group, COUNT(div);
DUMP letcount;

 div = FOREACH third_col_e GENERATE FLATTEN(TOKENIZE(c3)) AS c3;
 grouped = GROUP div BY c3;
 letcount = FOREACH grouped GENERATE group, COUNT(div);
 DUMP letcount;
2020-02-16 17:01:05,981 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 17:01:06,717 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 17:01:06,750 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 17:01:06,774 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 17:01:06,829 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2020-02-16 17:01:06,865 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1581861269524_0066
2020-02-16 17:01:06,884 [JobCon

In [5]:
%%pig
STORE letcount INTO 'output';

 STORE letcount INTO 'output';
2020-02-16 17:03:20,475 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2020-02-16 17:03:20,601 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 17:03:20,832 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 17:03:20,894 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 17:03:20,914 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 17:03:21,381 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2020-02-16 17:03:21,444 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for

In [6]:
%%pig
fs -get output/ .

 fs -get output/ .


In [7]:
!hadoop fs -rm output/*
!hadoop fs -rmdir output

Deleted output/_SUCCESS
Deleted output/part-r-00000


In [9]:
%pig_quit